In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, auc, roc_curve, roc_auc_score, balanced_accuracy_score, classification_report

In [6]:
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [7]:
dt = datetime.today()

In [8]:
def removeOutlier(dataSet, feature):    
    q1=dataSet[feature].quantile(0.25)
    q3=dataSet[feature].quantile(0.75)
    IQR=q3-q1
    lowerLimit = q1 - 1.5 * IQR
    UpperLimit = q3 + 1.5 * IQR 
    dataSet = dataSet[dataSet[feature]< UpperLimit]
    dataSet = dataSet[dataSet[feature]> lowerLimit]
    return dataSet

In [9]:
def modelEvaluation(model, X_test, y_train,y_test, y_pred,cols=None):
    print("Accurancy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    print("ROC AUC Score: {:.3f}".format(roc_auc_score(y_test, y_pred)))
    print("F1 Score:: {:.3f} ".format(f1_score(y_test, y_pred)))
    print("Balanced Accurancy Score:: {:.3f} ".format(balanced_accuracy_score(y_test, y_pred)))
    print('\n clasification report:\n', classification_report(y_test,y_pred))
    
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)
    prediction_probabilities = model.predict_proba(X_test[cols])[:,1]
    fpr , tpr , thresholds = roc_curve(y_test,prediction_probabilities)
    ax.plot(fpr,tpr,label = ["Area under curve : ",auc(fpr,tpr)],linewidth=2,linestyle="dotted")
    ax.plot([0,1],[0,1],linewidth=2,linestyle="dashed")
    plt.legend(loc="best")
    plt.title("ROC-CURVE and AREA UNDER CURVE")
    ax.set_facecolor("k")

In [10]:
def calculate_age(row):
    try:
        yrs, mon = tuple(row['AVERAGE_ACCT_AGE'].split(' '))
        age = round(float(yrs.replace('yrs', '')) + float(mon.replace('mon', ''))/12, 2)
        row['AVERAGE_ACCT_AGE'] = age

        yrs, mon = tuple(row['CREDIT_HISTORY_LENGTH'].split(' '))
        age = round(float(yrs.replace('yrs', '')) + float(mon.replace('mon', ''))/12, 2)
        row['CREDIT_HISTORY_LENGTH'] = age
    except Exception as e:
        print(row, e)
        raise e
    return row


In [11]:
def binning_by_depth_factor(df_column, factor):
    divs, max_da, min_da= round(np.sqrt(len(df_column))/factor), df_column.max(), df_column.min()
    step = (max_da - min_da)/divs
    return (df_column/step).astype(int)*divs


training_csv = '/kaggle/input/vehicle-loan-default-prediction/train.csv'
test_csv = '/kaggle/input/vehicle-loan-default-prediction/test.csv'

train_df = pd.read_csv(training_csv)
train_df = train_df[~train_df['LOAN_DEFAULT'].isna()]
print(train_df.columns)

train_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/vehicle-loan-default-prediction/train.csv'